In [1]:
%matplotlib inline
import os
import numpy as np
import sklearn as skl
import pickle
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC, LinearSVC
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.dummy import DummyClassifier
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from imblearn.under_sampling import RandomUnderSampler, OneSidedSelection
from imblearn.over_sampling import SMOTE
sns.set_style('darkgrid')

EXPORT_MODELS=True
IMPORT_MODELS=False
TESTING=False
if TESTING:
    IMPORT_MODELS = True
    EXPORT_MODELS = False


In [2]:
def load_model(model_path):
    model = None
    with open(str(model_path), 'rb') as handle:
        model = pickle.load(handle)
    return model

資料預處理

In [3]:
from sklearn.preprocessing import OneHotEncoder as OHE
from sklearn.preprocessing import LabelEncoder as LE
from sklearn.feature_extraction import DictVectorizer
train_path = './TraData.csv'
if not os.path.isdir('./feature_extractors'):
    os.makedirs('./feature_extractors')
if not os.path.isdir('./pca'):
    os.makedirs('./pca')
if not os.path.isdir('./models'):
    os.makedirs('./models')
data = pd.read_csv(train_path, sep=',') #.sample(100000) # toy
if not TESTING:
    if os.path.exists("./feature_extractors/le.pkl"):
        label_le = load_model("./feature_extractors/le.pkl")
    else:
        label_le = LE().fit(data.click)
        with open("./feature_extractors/le.pkl", "wb") as f: # export pca transformer
            pickle.dump(label_le, f, pickle.HIGHEST_PROTOCOL)
    label = label_le.transform(data.click)
    del data['click'] # 記得別讓答案變成一組 feature ，這樣 model 就直接看到答案了
# 特徵選擇、降維 改交給 SVD 分解完成
del data['ip'] # ip 的 dimension 有 70多萬維，太高了我們不要它
del data['spaceCat']
del data['adType']
# del data['os']
del data['deviceType']
del data['publisherId']
del data['dclkVerticals']
# del data['campaignId']
# del data['advertiserId']
if os.path.exists("./feature_extractors/dv.pkl"):
    dv = load_model("./feature_extractors/dv.pkl")
else:
    dv = DictVectorizer(sparse=False).fit(data.T.to_dict().values()) # 要執行這步，你/妳的 RAM 要夠大 (>8G 一定沒問題)
    with open("./feature_extractors/dv.pkl", "wb") as f: # export pca transformer
        pickle.dump(dv, f, pickle.HIGHEST_PROTOCOL)
data = dv.transform(data.T.to_dict().values())


In [4]:
print data[:3] ## 印出三筆資料觀察
print label[:3]

[[  2.66000000e+02   0.00000000e+00   1.00000000e+00   4.61200000e+03
    1.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00  

In [5]:
from sklearn.model_selection import train_test_split
if not TESTING:
    data, X_test, label, Y_test = train_test_split(data, np.asarray(label).flatten(), test_size=0.1)
    print data.shape
    print X_test.shape
    print label.shape
    print Y_test.shape

(865311, 329)
(96146, 329)
(865311,)
(96146,)


設定模型參數、建構模型

In [7]:
class_weight={0:1,1:16}
## DT on sklearn is based on CART algorithm. Let's try ID3&CART algorithm both. And SVM.
classifiers = {
                # 'SVC':SVC(kernel='rbf', class_weight=class_weight, max_iter=300), 
                #'LinearSVC':LinearSVC(class_weight=class_weight, max_iter=300),
                #'XGBoost': xgb.XGBClassifier(**param),
                'DecisionTree': tree.DecisionTreeClassifier(max_depth=5, class_weight=class_weight),
                'RandomForest': RandomForestClassifier(n_estimators=24, n_jobs=24, max_depth=5, class_weight=class_weight),
                # 'KNN_dist': KNN(n_neighbors=3, weights='distance',),
                # 'KNN_uni': KNN(n_neighbors=3, weights='uniform',),
                # SVM 複雜度太高了，算不完 O(n^3~n^4), 使用簡單的 LinearSVM(O(N))]
                # 'Random':DummyClassifier('uniform'), ## 下面這兩個是 baseline ，不應該比亂猜/都猜0還差
                # 'Guess0':DummyClassifier('most_frequent')
              }

In [8]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

In [9]:
if IMPORT_MODELS==True and os.path.exists("./pca/pca.pkl"):
    svd = load_model("./pca/pca.pkl")
else:
    # 要執行這步，你/妳的 RAM 要夠大 (>8G 一定沒問題)
    svd = PCA(n_components=100).fit(data) # 降維，維度太高會發生'維度災難'
    with open("./pca/pca.pkl", "wb") as f: # export pca transformer
        pickle.dump(svd, f, pickle.HIGHEST_PROTOCOL)

In [10]:
svd_data = svd.transform(data)
print svd_data.shape
print np.cumsum(svd.explained_variance_ratio_)
print 'info: %.2f'%np.sum(svd.explained_variance_ratio_)
print 'nans: %d'%np.sum(np.isnan(svd_data))

(865311, 100)
[ 0.89889502  0.99970436  0.99977868  0.99982866  0.99985944  0.99987677
  0.99988997  0.99990118  0.99991076  0.99991848  0.99992487  0.99993037
  0.99993476  0.99993822  0.99994114  0.99994367  0.99994598  0.99994822
  0.99995044  0.99995261  0.99995467  0.99995658  0.99995833  0.99996001
  0.99996157  0.99996294  0.99996417  0.9999653   0.99996634  0.99996732
  0.99996829  0.99996925  0.99997015  0.99997102  0.99997188  0.99997273
  0.99997357  0.99997441  0.99997525  0.99997606  0.99997686  0.99997763
  0.99997838  0.99997913  0.99997983  0.99998051  0.99998119  0.99998183
  0.99998244  0.99998302  0.99998359  0.99998416  0.99998472  0.99998528
  0.99998583  0.99998638  0.99998689  0.99998738  0.99998785  0.99998829
  0.99998871  0.99998912  0.99998953  0.99998992  0.99999026  0.99999058
  0.99999089  0.99999118  0.99999147  0.99999176  0.99999203  0.9999923
  0.99999256  0.99999282  0.99999307  0.99999332  0.99999355  0.99999378
  0.99999399  0.9999942   0.9999944   

In [11]:
data = svd_data
del svd_data
models = []

10-fold Cross-validation

In [12]:
if not TESTING:    
    kfold = StratifiedKFold(n_splits=10, shuffle=True)
    models = []
    cvscores = []
    for model_name, model in classifiers.items():
        print('Training %s...'%model_name)
        tempscores = []
        for train, test in kfold.split(data, label):
            X, y = data[train], label[train]
            model.fit(X,y)
            pred = model.predict(data[test])
            pred[pred<0.5] = 0
            pred[pred>0] = 1
            score = f1_score(label[test], pred, average='binary')
            tempscores.append(score)
            models.append((str(model_name), model, score))
        print("f1.avg: %.4f%% (+/- %.4f%%)" % (np.mean(tempscores)*100., np.std(tempscores)*100.)) ## validation 的 f1-score 的平均值 +/- 兩倍標準差
        cvscores.extend(tempscores)

Training DecisionTree...
f1.avg: 16.3478% (+/- 3.0472%)
Training XGBoost...
f1.avg: 16.8683% (+/- 2.7672%)
Training RandomForest...
f1.avg: 15.7667% (+/- 3.9390%)


In [13]:
if IMPORT_MODELS==True:
    model_dir = './models'
    model_list = os.listdir(model_dir)
    for filename in model_list:
        full_path = model_dir+'/'+filename
        score = float((''.join(filename.split('.')[:-1])).split('-')[-1])
        model = load_model(str(full_path))
        models.append((filename, model, score))

In [14]:
if EXPORT_MODELS==True:
    for i, (model_name, model, score) in enumerate(models):
        with open("./models/%02d-%s-%.2f.pkl"%(i, str(model_name), score), "wb") as f:
            pickle.dump(model, f, pickle.HIGHEST_PROTOCOL)

In [16]:
def voter(models, data, tol=0.12): # set tolerance to ignore weak classifier
    final_p = np.zeros(len(data))
    for i, entry in enumerate(models):
        model, score = entry[-2], entry[-1]
        if score<tol: continue
        pred = (np.asarray(model.predict(data)).flatten()-0.5)*score
        final_p += pred # 有權重的投票
    final_p[final_p<0] = 0
    final_p[final_p>0] = 1
    return final_p

In [25]:
from sklearn.model_selection import train_test_split
if not TESTING:
    X_test_svd = svd.transform(X_test)
    predicted = voter(models, X_test_svd, tol=0.12)
    confusion_metrix = skl.metrics.confusion_matrix(Y_test, predicted)
    inclass_precision = skl.metrics.classification_report(Y_test, predicted)
    score = f1_score(Y_test, predicted, average='binary')
    print(confusion_metrix)
    print(inclass_precision)
    print(score)
else:
    predicted = voter(models, data).flatten()
    with open('results_label.txt', 'w') as fp:
        fp.write('click\n') # field name
        for v in predicted:
            fp.write('%d\n'%int(v))

[[96002    79]
 [   51    14]]
             precision    recall  f1-score   support

          0       1.00      1.00      1.00     96081
          1       0.15      0.22      0.18        65

avg / total       1.00      1.00      1.00     96146

0.177215189873


補充從 R script 得到的這份 training data 的 ANOVA 結果（從九十萬筆資料隨機抽樣5000筆資料）：

```R
               Df Sum Sq  Mean Sq   F value Pr(>F)    
adx             1  0.000 0.000003 2.192e+23 <2e-16 ***
spaceType       1  0.000 0.000011 9.356e+23 <2e-16 ***
spaceId       128  0.516 0.004034 3.451e+26 <2e-16 ***
ip           4839  3.480 0.000719 6.154e+25 <2e-16 ***
os              3  0.000 0.000000 2.000e-03 0.9999    
campaignId      1  0.000 0.000000 3.089e+00 0.0911 .  
advertiserId    1  0.000 0.000000 1.496e+00 0.2327    
Residuals      25  0.000 0.000000                     
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1
```

可以看出 adx, spaceType, spaceId, ip 對 click 的影響似乎是顯著的？

也許可以實驗看看只選擇 adx, spaceType, spaceId, ip 當作 feature？或是使用 PCA 做降維？

因為他們對 click 的影響較顯著？ p<0.05